In [61]:
options(warn=-1)
library(ggplot2)

library(motifmatchr)

library(SummarizedExperiment)
library(Matrix)

library(BiocParallel)
library(BSgenome.Hsapiens.UCSC.hg19)

library(data.table)
library(parallel)
library(scABC)

library(chromVAR)
library(JASPAR2016)

set.seed(1234)

# Load data

In [62]:
# Load data
data_path <- "/home/sccasimp/data/DownStreamAnalysis/"
for(chr in 1:22){
  if(chr==1){
    # https://doi.org/10.5281/zenodo.7768714 -> 1000G_Phase3_plinkfiles.tgz
    bim <- fread(paste0(data_path, '1000G_EUR_Phase3_plink/1000G.EUR.QC.1.bim'))
  }else{
    bim <- rbind(bim,fread(paste0(data_path, '1000G_EUR_Phase3_plink/1000G.EUR.QC.',chr,'.bim')))
  }
} ## 1000G bim file
bim$V1 <- paste0('chr',bim$V1)

In [ ]:
bim[1:5]

In [38]:
find.index <- function(df1,df2,type='reg'){
  #colnames(df1) <- colnames(df2) <- c('V1','V2','V3')
  library(GenomicRanges)
  df1.gr = GRanges (IRanges(start = df1$V2, end = df1$V3), seqnames=df1$V1) 
  if(type=='reg'){
    df2.gr = GRanges(IRanges(start=df2$V2, end = df2$V3), seqnames = df2$V1) 
  }
  if(type=='pos'){
    df2.gr = GRanges(IRanges(start=df2$V4, end = df2$V4), seqnames = df2$V1) 
  }
  df1.git  = GNCList(df1.gr)
  df2.git  = GNCList(df2.gr)
  overlap_git = findOverlaps(df2.git, df1.git)
  overlap_git
  temp <- as.data.frame(overlap_git)
  colnames(temp) <- c('df2','df1')
  return(temp)
}

# Expression enrichment analysis

In [63]:
inputpath <- "./ATAC2RNA/pred/"
outputpath <- "./ATAC2RNA/snpsea/"

In [64]:
## 1. bg
name <- c('bg3')
for(k in 1:1){
  print(name[k])
  peak <- fread(paste0(inputpath,name[k],'.bed'))
  ind.temp <- find.index(peak,bim,type='pos')
  bim1 <- bim[ind.temp$df2,c(1,4,2)]
  colnames(bim1) <- c("CHR","POS","SNP")
  print(dim(bim1))
  write.table(bim1,paste0(outputpath,'anno/',gsub(" ","_",name[k]),'.anno'),sep="\t",quote=F,col.names=T,row.names=F)
}

[1] "bg3"
[1] 4390    3


In [65]:
## 2. specific annotations
name <- c('CD14_Mono',
 'NK',
 'Treg',
 'CD8_Naive',
 'MAIT',
 'CD16_Mono',
 'CD4_Naive',
 'CD4_TCM',
 'CD8_TEM_2',
 'CD8_TEM_1',
 'Intermediate_B',
 'cDC',
 'gdT',
 'CD4_TEM',
 'HSPC',
 'Memory_B',
 'Naive_B',
 'pDC',
 'Plasma')
for(k in 1:22){
  print(name[k])
  peak <- fread(paste0(inputpath,name[k],'.bed'))
  ind.temp <- find.index(peak,bim,type='pos')
  bim1 <- bim[ind.temp$df2,c(1,4,2)]
  colnames(bim1) <- c("CHR","POS","SNP")
  print(dim(bim1))
  write.table(bim1,paste0(outputpath,'anno/',gsub(" ","_",name[k]),'.anno'),sep="\t",quote=F,col.names=T,row.names=F)
}

[1] "CD14_Mono"
[1] 3384    3
[1] "NK"
[1] 6198    3
[1] "Treg"
[1] 10606     3
[1] "CD8_Naive"
[1] 3046    3
[1] "MAIT"
[1] 8734    3
[1] "CD16_Mono"
[1] 2638    3
[1] "CD4_Naive"
[1] 3333    3
[1] "CD4_TCM"
[1] 5599    3
[1] "CD8_TEM_2"
[1] 6037    3
[1] "CD8_TEM_1"
[1] 11914     3
[1] "Intermediate_B"
[1] 4781    3
[1] "cDC"
[1] 4252    3
[1] "gdT"
[1] 8585    3
[1] "CD4_TEM"
[1] 7203    3
[1] "HSPC"
[1] 5426    3
[1] "Memory_B"
[1] 3748    3
[1] "Naive_B"
[1] 5159    3
[1] "pDC"
[1] 4808    3
[1] "Plasma"
[1] 5895    3
[1] NA


ERROR: Error in fread(paste0(inputpath, name[k], ".bed")): File './ATAC2RNA/pred/NA.bed' does not exist or is non-readable. getwd()=='/work/cabins/sccasimp/software'


In [ ]:
## perform snpsea
## download necessary files from: 
# **Home Page:** <http://www.broadinstitute.org/mpg/snpsea

>
# **Executable:** [snpsea-v1.0.3.tar.gz][exec]
# **Data:** [SNPsea_data_20140520.zip][data]

In [17]:
# # shell
# options=(
#     --snps              /home/sccasimp/program/sccasimp/revision/DA/anno/bg2.anno
#     --gene-matrix       /home/sccasimp/data/DownStreamAnalysis/GeneAtlas2004.gct.gz
#     --gene-intervals    /home/sccasimp/data/DownStreamAnalysis/NCBIgenes2013.bed.gz
#     --snp-intervals     /home/sccasimp/data/DownStreamAnalysis/TGP2011.bed.gz
#     --null-snps         /home/sccasimp/data/DownStreamAnalysis/Lango2010.txt.gz
#     --out               /home/sccasimp/program/sccasimp/revision/DA/anno/bg2.anno.out
#     --slop              10e3
#     --threads           4
#     --null-snpsets      0
#     --min-observations  100
#     --max-iterations    1e7
# )
# /home/sccasimp/software/snpsea_v1.0.2/bin/snpsea ${options[*]}
# /home/sccasimp/software/snpsea_v1.0.2/bin/snpsea-barplot_modified /home/sccasimp/program/sccasimp/revision/DA/anno/bg2.anno.out --top 30 --fontsize 10

ERROR: Error in parse(text = x, srcfile = src): <text>:9:25: unexpected numeric constant
8:     --out               /home/sccasimp/program/sccasimp/revision/DA/anno/bg.anno.out
9:     --slop              10e3
                           ^


# Partitioned heritability analysis

In [7]:
# # data example
# SNP	A1	A2	N	CHISQ	Z
# rs3094315	G	A	443940	0.0138	0.1174
# rs3131972	A	G	443940	0.0142	0.1192
# rs3131969	A	G	443940	0.8643	0.9297
# rs1048488	C	T	443940	0.0434	0.2084
# rs3115850	T	C	443940	0.0417	0.2043
# rs2286139	C	T	443940	0.4812	0.6937
# rs12562034	G	A	443940	0.0546	0.2336
# rs4040617	A	G	443940	0.5676	-0.7534
# rs2980300	T	C	443940	0.4392	0.6627

ERROR: Error in parse(text = x, srcfile = src): <text>:2:9: unexpected symbol
1: # data example
2: SNP     A1
           ^


In [66]:
inputpath <- "./ATAC2RNA/pred/"
outputpath <- "./ATAC2RNA/"

In [67]:
 ## 1000G bim file
bg_name <- 'bg3'
bg1 <- fread(paste0(inputpath,bg_name,'.bed'))
ind.bg <- find.index(bg1,bim,type='pos')
snpset <- list()
snpset[[1]] <- bim$V2[ind.bg$df2]

In [68]:
peak <- list()
name <-c('CD14_Mono',
 'NK',
 'Treg',
 'CD8_Naive',
 'MAIT',
 'CD16_Mono',
 'CD4_Naive',
 'CD4_TCM',
 'CD8_TEM_2',
 'CD8_TEM_1',
 'Intermediate_B',
 'cDC',
 'gdT',
 'CD4_TEM',
 'HSPC',
 'Memory_B',
 'Naive_B',
 'pDC',
 'Plasma')
for(k in 1:length(name)){
  peak[[k]] <- fread(paste0(inputpath,name[k],'.bed'))
  ind.temp <- find.index(peak[[k]],bim,type='pos')
  snpset[[k+1]] <- bim$V2[ind.temp$df2]
}

In [69]:
for(chr in 1:22){
  print(chr)
  bim2 <- fread(paste0(data_path, '1000G_EUR_Phase3_plink/1000G.EUR.QC.',chr,'.bim')) ## 1000G bim file for each chromosome
  for(j in 1:(length(name)+1)){
    index <- which(bim2$V2%in%snpset[[j]])
    anno <- rep(0,nrow(bim2))
    anno[index] <- 1
    if(j==1){
      anno1 <- cbind(rep(1,nrow(bim2)),anno)
    }else{
      anno1 <- cbind(anno1,anno)
    }
  }
  colnames(anno1) <- c('base',bg_name,name)
  write.table(anno1,paste0(outputpath,'LDSC/epi.mm',chr,'.annot'),quote=F,row.names=F,col.names=T,sep='\t')
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22


In [39]:
# shell
# summarystats

for i in {1..22}
do python /home/sccasimp/software/ldsc/ldsc.py --l2 --bfile /home/sccasimp/data/DownStreamAnalysis/1000G_EUR_Phase3_plink/1000G.EUR.QC.$i --ld-wind-cm 1 --annot /home/sccasimp/software/ATAC2RNA/LDSC/epi.mm$i.annot --thin-annot --out /home/sccasimp/software/ATAC2RNA/LDSC/epi.mm$i
done


for name in `ls /home/sccasimp/data/DownStreamAnalysis/sumstats/*.gz`;
do 
echo ${name}
python /home/sccasimp/software/ldsc/ldsc.py --h2 ${name} --ref-ld-chr /home/sccasimp/software/ATAC2RNA/LDSC/epi.mm --w-ld-chr /home/sccasimp/software/ldsc/eur_w_ld_chr/weights.hm3_noMHC. --overlap-annot --not-M-5-50 --out /home/sccasimp/software/ATAC2RNA/LDSC/${name#*sumstats/}
done

ERROR: Error in parse(text = x, srcfile = src): <text>:3:5: unexpected symbol
2: # summarystats
3: for name
       ^
